# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../da…",0.983033,"""81701663760121…","""17181442165297…","""27836849224021…","""17887856683503…",false
"""../../../../da…",0.144221,"""12094951921698…","""18848308573589…","""27836849224021…","""58499450088185…",true
"""../../../../da…",0.679745,"""31518682012029…","""15512005559854…","""12258085065076…","""17887856683503…",false
"""../../../../da…",1.0,"""12094951921698…","""33215299827652…","""95060435695362…","""17887856683503…",false
"""../../../../da…",0.718982,"""81701663760121…","""12042592091914…","""95060435695362…","""12483061181518…",true
"""../../../../da…",0.0,"""31518682012029…","""11300891304573…","""74067613040551…",null,false
"""../../../../da…",0.02333,"""31518682012029…","""94335802046083…","""95060435695362…","""12483061181518…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""11092292457853…","""50341947693815…","""FirstMortgage3…","""48187139684324…","""Rejected"""
"""../../../../da…",0.144221,"""44750233197729…","""16182007178025…","""FirstMortgage3…","""18065876345323…","""Accepted"""
"""../../../../da…",0.679745,"""11206293772999…","""56415102783961…","""MoneyMarketSav…","""48187139684324…","""Rejected"""
"""../../../../da…",1.0,"""44750233197729…","""14777679553099…","""BasicChecking""","""48187139684324…","""Rejected"""
"""../../../../da…",0.718982,"""11092292457853…","""17544931583175…","""BasicChecking""","""16735609960521…","""Accepted"""
"""../../../../da…",0.0,"""11206293772999…","""23209049094547…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""11206293772999…","""13528634905035…","""BasicChecking""","""16735609960521…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""21857111409190…","""58501827839619…","""FirstMortgage3…","""96345335742264…","""Rejected"""
"""../../../../da…",0.144221,"""15929699332520…","""16411158005903…","""FirstMortgage3…","""21278487074756…","""Accepted"""
"""../../../../da…",0.679745,"""93119567777356…","""84785521362207…","""MoneyMarketSav…","""96345335742264…","""Rejected"""
"""../../../../da…",1.0,"""15929699332520…","""11291379350897…","""BasicChecking""","""96345335742264…","""Rejected"""
"""../../../../da…",0.718982,"""21857111409190…","""13633485781360…","""BasicChecking""","""16091929067385…","""Accepted"""
"""../../../../da…",0.0,"""93119567777356…","""10631640587607…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""93119567777356…","""71873878467237…","""BasicChecking""","""16091929067385…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
0.983033,"""13686919398513…","""72959366638869…","""FirstMortgage3…","""72566773098919…","""Rejected"""
0.144221,"""86438975486424…","""40668026573728…","""FirstMortgage3…","""66444887132843…","""Accepted"""
0.679745,"""14309242750737…","""91133572366819…","""MoneyMarketSav…","""72566773098919…","""Rejected"""
1.0,"""86438975486424…","""73109584135671…","""BasicChecking""","""72566773098919…","""Rejected"""
0.718982,"""13686919398513…","""69253245190037…","""BasicChecking""","""71536781826709…","""Accepted"""
0.0,"""14309242750737…","""62096544634083…","""UPlusFinPerson…",null,"""Rejected"""
0.02333,"""14309242750737…","""98349458799446…","""BasicChecking""","""71536781826709…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",0.983033,"""af486bc1815c72…","""c73023dda51de1…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",0.144221,"""3cdf7b1ae1342e…","""5414b197fb87b3…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""../../../../da…",0.679745,"""806ef34eb8237f…","""892f5d61a9215e…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",1.0,"""3cdf7b1ae1342e…","""c3aa807d9e927a…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…",0.718982,"""af486bc1815c72…","""c2d492f9a05073…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""../../../../da…",0.0,"""806ef34eb8237f…","""b21059221605c4…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",0.02333,"""806ef34eb8237f…","""80cbe489dcecd4…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
